In [3]:
import pandas as pd
import numpy as np
import time
import sqlite3

data_home = '/Users/jgu053/Downloads/million-master/data/origin/'

In [4]:
triplet_dataset = pd.read_csv(filepath_or_buffer=data_home+'train_triplets.txt',
                              sep='\t', header=None,
                              names=['user', 'song', 'play_count'])

In [8]:
# Make the play_count column int32. save memory
triplet_dataset['play_count'] = triplet_dataset['play_count'].astype('int32')

In [9]:
triplet_dataset.shape

(48373586, 3)

In [10]:
triplet_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48373586 entries, 0 to 48373585
Data columns (total 3 columns):
user          object
song          object
play_count    int32
dtypes: int32(1), object(2)
memory usage: 922.7+ MB


In [11]:
triplet_dataset.head(n=10)

,user,song,play_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1
5,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBNZDC12A6D4FC103,1
6,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBSUJE12A6D4F8CF5,2
7,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBVFZR12A6D4F8AE3,1
8,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXALG12A8C13C108,1
9,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1


In [29]:
# 统计每个用户的播放总量
output_dict = {}
with open(data_home+'train_triplets.txt') as f:
    for i, line in enumerate(f):
        user = line.split('\t')[0]
        play_count = int(line.split('\t')[2])
        if user in output_dict:
            play_count += output_dict[user]
            output_dict.update({ user: play_count })
        output_dict.update({user:play_count})
        
output_list = [{'user':k, 'play_count':v} for k, v in output_dict.items()]
play_count_df = pd.DataFrame(output_list)
play_count_df = play_count_df.sort_values(by = 'play_count', ascending=False)

In [30]:
play_count_df.to_csv(path_or_buf='user_playcount_df.csv', index=False)

In [25]:
# 统计每首歌的总播放量
output_dict = {}
with open(data_home+'train_triplets.txt') as f:
    for i, line in enumerate(f):
        song = line.split('\t')[1]
        play_count = int(line.split('\t')[2])
        if song in output_dict:
            play_count += output_dict[song]
            output_dict.update({ song: play_count })
        output_dict.update({song:play_count})
        
output_list = [{'song': k, 'play_count': v} for k, v in output_dict.items()]
play_count_df = pd.DataFrame(output_list)
play_count_df = play_count_df.sort_values(by = 'play_count', ascending=False)

In [26]:
play_count_df.to_csv(path_or_buf='song_playcount_df.csv', index=False)

In [32]:
user_count_df = pd.read_csv('user_playcount_df.csv')
user_count_df.head(n=10)

,play_count,user
0,13132,093cb74eb3c517c5179ae24caf0ebec51b24d2a2
1,9884,119b7c88d58d0c6eb051365c103da5caf817bea6
2,8210,3fa44653315697f42410a30cb766a4eb102080bb
3,7015,a2679496cd0af9779a92a13ff7c6af5c81ea8c7b
4,6494,d7d2d888ae04d16e994d6964214a1de81392ee04
5,6472,4ae01afa8f2430ea0704d502bc7b57fb52164882
6,6150,b7c24f770be6b802805ac0e2106624a517643c17
7,5656,113255a012b2affeab62607563d03fbdf31b08e7
8,5620,6d625c6557df84b60d90426c0116138b617b9449
9,5602,99ac3d883681e21ea68071019dba828ce76fe94d


In [33]:
song_count_df = pd.read_csv('song_playcount_df.csv')
song_count_df.head(n=10)

,play_count,song
0,726885,SOBONKR12A58A7A7E0
1,648239,SOAUWYT12A81C206F1
2,527893,SOSXLTC12AF72A7F54
3,425463,SOFRQTD12A81C233C0
4,389880,SOEGIYH12A6D4FC0E3
5,356533,SOAXGDH12A8C13F8A1
6,292642,SONYKOW12AB01849C9
7,274627,SOPUCYA12A8C13A694
8,268353,SOUFTBI12AB0183F65
9,244730,SOVDSJC12A58A7A271


In [34]:
print(len(user_count_df))

1019318


In [49]:
total_play_count = sum(song_count_df.play_count)

# 300k 用户，播放量占总总量71.33%
float(user_count_df.head(n=150000).play_count.sum() / total_play_count) * 100

51.1606033167969

In [50]:
# 30k歌曲，播放总量占78.39%
float(song_count_df.head(n=30000).play_count.sum() / total_play_count) * 100

78.39315366645269

In [60]:
# 取top 300k首歌, top 30k users
user_count_subset = user_count_df.head(n=150000)
song_count_subset = song_count_df.head(n=30000)

In [61]:
# 取top 300k user
user_subset = list(user_count_subset.user)
song_subset = list(song_count_subset.song)

In [62]:
# Filter data into new csv files
triplet_dataset = pd.read_csv(data_home + 'train_triplets.txt', sep='\t',
                             header=None, names=['user', 'song', 'play_count'])
triplet_dataset_sub = triplet_dataset[triplet_dataset.user.isin(user_subset)]
del(triplet_dataset)
triplet_dataset_sub_song = triplet_dataset_sub[triplet_dataset_sub.song.isin(song_subset)]
del(triplet_dataset_sub)

In [63]:
triplet_dataset_sub_song.to_csv(data_home + 'triplet_dataset_sub_song.csv', index=False)

In [64]:
triplet_dataset_sub_song.shape

(14170027, 3)

In [65]:
conn = sqlite3.connect(data_home + 'track_metadata.db')
cur = conn.cursor()
cur.execute('select name from sqlite_master where type="table"')
cur.fetchall()

[('songs',)]

In [66]:
track_metadata_df = pd.read_sql(con=conn, sql='select * from songs')
track_metadata_df_sub = track_metadata_df[track_metadata_df.song_id.isin(song_subset)]

In [67]:
track_metadata_df_sub.to_csv(data_home+'track_metadata_df_sub.csv', index=False)

In [68]:
track_metadata_df_sub.shape

(30447, 14)

In [69]:
# Now we have:
triplet_dataset_sub_song = pd.read_csv(data_home+'triplet_dataset_sub_song.csv', encoding='ISO-8859-1')
track_meta_df_sub = pd.read_csv(data_home+'track_metadata_df_sub.csv', encoding='ISO-8859-1')

In [72]:
track_meta_df_sub.head()

,title,song_id,release,artist_id,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,Get Along (Feat: Pace Won) (Instrumental),SOHNWIM12A67ADF7D9,Charango,ARU3C671187FB3F71B,Morcheeba,227.47383,0.819087,0.533117,2002,185967,-1,0
1,Viejo,SOECFIW12A8C144546,Caraluna,ARPAAPH1187FB3601B,Bacilos,307.51302,0.595554,0.400705,0,6825058,-1,0
2,I Say A Little Prayer,SOGWEOB12AB018A4D0,The Legendary Hi Records Albums_ Volume 3: Ful...,ARNNRN31187B9AE7B7,Al Green,133.58975,0.779490,0.599210,1978,5211723,-1,11898
3,At the Ball_ That's All,SOJGCRL12A8C144187,Best of Laurel & Hardy - The Lonesome Pine,AR1FEUF1187B9AF3E3,Laurel & Hardy,123.71546,0.438709,0.307120,0,8645877,-1,0
4,Black Gold,SOHNFBA12AB018CD1D,Total Life Forever,ARVXV1J1187FB5BF88,Foals,386.32444,0.842578,0.514523,2010,9007438,-1,0


In [71]:
del(track_meta_df_sub['track_id'])
del(track_meta_df_sub['artist_mbid'])
track_meta_df_sub = track_meta_df_sub.drop_duplicates(['song_id'])

In [73]:
triplet_dataset_sub_song_merged = pd.merge(triplet_dataset_sub_song, track_meta_df_sub, how='left', left_on='song', right_on='song_id')
triplet_dataset_sub_song_merged.rename(columns={'play_count':'listen_count'}, inplace=True)

In [74]:
triplet_dataset_sub_song_merged.head()

,user,song,listen_count,title,song_id,release,artist_id,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOADQPP12A67020C82,12,You And Me Jesus,SOADQPP12A67020C82,Tribute To Jake Hess,ARDFZE61187FB45994,Jake Hess,199.26159,0.374553,0.327544,2004,142275,-1,0
1,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAFTRR12AF72A8D4D,1,Harder Better Faster Stronger,SOAFTRR12AF72A8D4D,Discovery,ARF8HTQ1187B9AE693,Daft Punk,223.60771,0.902841,1.021256,2007,485119,-1,0
2,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOANQFY12AB0183239,1,Uprising,SOANQFY12AB0183239,Uprising,ARR3ONV1187B9A2F59,Muse,304.84853,0.929030,0.750427,0,6024257,-1,0
3,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAYATB12A6701FD50,1,Breakfast At Tiffany's,SOAYATB12A6701FD50,Home,AR97V5X1187FB4AE98,Deep Blue Something,257.22730,0.655889,0.409729,1993,163047,-1,0
4,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBOAFP12A8C131F36,7,Lucky (Album Version),SOBOAFP12A8C131F36,We Sing. We Dance. We Steal Things.,ARML3X41187FB35F2E,Jason Mraz & Colbie Caillat,189.62240,0.842972,0.770117,0,2638597,-1,0


In [75]:
del(triplet_dataset_sub_song_merged['song_id'])
del(triplet_dataset_sub_song_merged['artist_id'])
del(triplet_dataset_sub_song_merged['duration'])
del(triplet_dataset_sub_song_merged['artist_familiarity'])
del(triplet_dataset_sub_song_merged['artist_hotttnesss'])
del(triplet_dataset_sub_song_merged['track_7digitalid'])
del(triplet_dataset_sub_song_merged['shs_perf'])
del(triplet_dataset_sub_song_merged['shs_work'])

In [76]:
triplet_dataset_sub_song_merged.head()

,user,song,listen_count,title,release,artist_name,year
0,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOADQPP12A67020C82,12,You And Me Jesus,Tribute To Jake Hess,Jake Hess,2004
1,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAFTRR12AF72A8D4D,1,Harder Better Faster Stronger,Discovery,Daft Punk,2007
2,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOANQFY12AB0183239,1,Uprising,Uprising,Muse,0
3,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAYATB12A6701FD50,1,Breakfast At Tiffany's,Home,Deep Blue Something,1993
4,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBOAFP12A8C131F36,7,Lucky (Album Version),We Sing. We Dance. We Steal Things.,Jason Mraz & Colbie Caillat,0


# Start recommend part

In [88]:
import Recommenders as Recommenders
from sklearn.model_selection import train_test_split

### 1. Easy way

Recommend most popular items

In [85]:
triplet_dataset_song_merged_set.head(5)

,user,song,listen_count,title,release,artist_name,year
0,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOADQPP12A67020C82,12,You And Me Jesus,Tribute To Jake Hess,Jake Hess,2004
1,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAFTRR12AF72A8D4D,1,Harder Better Faster Stronger,Discovery,Daft Punk,2007
2,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOANQFY12AB0183239,1,Uprising,Uprising,Muse,0
3,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAYATB12A6701FD50,1,Breakfast At Tiffany's,Home,Deep Blue Something,1993
4,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBOAFP12A8C131F36,7,Lucky (Album Version),We Sing. We Dance. We Steal Things.,Jason Mraz & Colbie Caillat,0


In [89]:
triplet_dataset_song_merged_set = triplet_dataset_sub_song_merged
train_data, test_data = train_test_split(triplet_dataset_song_merged_set, test_size=0.40, random_state=0)

In [91]:
train_data.head()

,user,song,listen_count,title,release,artist_name,year
4503805,8811f711d0d269c4ea9c03bcedd6d2dc1b9f7a64,SOASPCN12A6D4F8589,1,Recess,Hullabaloo Soundtrack,Muse,0
4938144,261c4827477fccfe5cedfcc3c66738845a66a596,SOSUBHV12AF72A4220,4,No I In Threesome,Our Love To Admire,Interpol,0
6246435,45bf3ef97843d6175020ed2f8c915e3677e48e7d,SOWSEHM12A8C131290,37,No Children,Tallahassee,Mountain Goats,2002
11347726,8e742f06033156624a65cc5ba36c3add38b060f8,SOHWXEU12A8C1320D1,1,Dans Ma Bulle (Edit Radio - Live 2006),Dans Ma Bulle,Diam's,2006
6577510,d9b6da92dec914c16630b91ca10909342813e6b3,SOOAVGC12AB01821EC,1,The Bachelor and the Bride,Her Majesty The Decemberists,The Decemberists,2003


In [94]:
def create_popularity_recommendation(train_data, sortby='user', groupby='title'):
    # Get a count of user_ids for each unique song as recommendation score
    train_data_grouped = train_data.groupby([groupby]).agg({sortby: 'count'}).reset_index()
    train_data_grouped.rename(columns= {sortby: 'score'}, inplace=True)
    
    # Sort the songs based upon recommendation score
    train_data_sort = train_data_grouped.sort_values(['score', groupby], ascending=[0,1])
    
    # Generate a recommendation rank based upon score
    train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first')
    
    # Get the top 10 recommendations
    popularity_recommendations = train_data_sort.head(20)
    return popularity_recommendations

In [95]:
recommendations = create_popularity_recommendation(train_data, 'user', 'title')

In [96]:
recommendations

,title,score,Rank
19580,Sehr kosmisch,15895,1.0
5780,Dog Days Are Over (Radio Edit),14939,2.0
27314,You're The One,13526,3.0
19542,Secrets,12969,4.0
25070,Undo,12716,5.0
18636,Revelry,12664,6.0
7530,Fireflies,10946,7.0
9640,Hey_ Soul Sister,10877,8.0
25216,Use Somebody,10424,9.0
9921,Horn Concerto No. 4 in E flat K495: II. Romanc...,10389,10.0


### 2. Recommend based on similarity (correlation)

先用小部分数据做个实验：

In [98]:
song_count_subset = song_count_df.head(n=5000)
user_subset = list(user_count_subset.user)
song_subset = list(song_count_subset.song)
triplet_dataset_sub_song_merged_sub = triplet_dataset_sub_song_merged[triplet_dataset_sub_song_merged.song.isin(song_subset)]


In [99]:
triplet_dataset_sub_song_merged_sub.head()

,user,song,listen_count,title,release,artist_name,year
0,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOADQPP12A67020C82,12,You And Me Jesus,Tribute To Jake Hess,Jake Hess,2004
1,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAFTRR12AF72A8D4D,1,Harder Better Faster Stronger,Discovery,Daft Punk,2007
2,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOANQFY12AB0183239,1,Uprising,Uprising,Muse,0
3,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOAYATB12A6701FD50,1,Breakfast At Tiffany's,Home,Deep Blue Something,1993
4,d6589314c0a9bcbca4fee0c93b14bc402363afea,SOBOAFP12A8C131F36,7,Lucky (Album Version),We Sing. We Dance. We Steal Things.,Jason Mraz & Colbie Caillat,0


In [ ]:
train_data, 